In [2]:
# Imports
import pyomo.environ as pyomo
import numpy as np

In [3]:
# Reading problem instance from file 
def read_problem_instance(file_path):
    with open(file_path, "r") as file:
        lines = file.readlines()

    # Parse data
    num_facilities = int(lines[1].split(': ')[1])
    num_customers = int(lines[2].split(': ')[1])
    grid_size = int(lines[3].split(': ')[1].split('x')[0])

    facilities_coords = []
    customers_coords = []
    distances = np.zeros((num_facilities, num_customers))
    cost_s = np.zeros((num_facilities, num_customers), dtype=int)
    cost_l = np.zeros((num_facilities, num_customers), dtype=int)
    facility_capacities = []
    customer_weights = []  # List to store customer weights

    line_idx = 5  # Start parsing after the grid size

    # Facilities coordinates
    for _ in range(num_facilities):
        x, y = map(int, lines[line_idx].split(','))
        facilities_coords.append((x, y))
        line_idx += 1

    # Customers coordinates
    line_idx += 1
    for _ in range(num_customers):
        x, y = map(int, lines[line_idx].split(','))
        customers_coords.append((x, y))
        line_idx += 1

    # Distances
    line_idx += 1
    for i in range(num_facilities):
        for j in range(num_customers):
            distances[i][j] = float(lines[line_idx].split(': ')[1])
            line_idx += 1

    # Costs for S-type drones
    line_idx += 1
    for i in range(num_facilities):
        for j in range(num_customers):
            cost_s[i][j] = int(lines[line_idx].split(': ')[1])
            line_idx += 1

    # Costs for L-type drones
    line_idx += 1
    for i in range(num_facilities):
        for j in range(num_customers):
            cost_l[i][j] = int(lines[line_idx].split(': ')[1])
            line_idx += 1

    # Facility capacities
    line_idx += 1
    for _ in range(num_facilities):
        facility_capacities.append(int(lines[line_idx].split(': ')[1]))
        line_idx += 1

    # Small and large drone capacities and ranges
    payload_small = int(lines[line_idx].split(': ')[1])
    payload_large = int(lines[line_idx + 1].split(': ')[1])
    range_small = int(lines[line_idx + 2].split(': ')[1])
    range_large = int(lines[line_idx + 3].split(': ')[1])

    # Check if customer weights section exists
    if line_idx + 4 < len(lines) and lines[line_idx + 4].startswith("Package weights"):
        # Read customer weights if the section exists
        line_idx += 5  # Skip to the section after the drone capacities and ranges
        for _ in range(num_customers):
            customer_weight = int(lines[line_idx].split(': ')[1])
            customer_weights.append(customer_weight)
            line_idx += 1
    else:
        # If no package weights section, assign default weights or handle error
        print("Warning: No customer weights found. Using default weights.")
        customer_weights = [1] * num_customers  # Assign default weight of 1

    return {
        "num_facilities": num_facilities,
        "num_customers": num_customers,
        "distances": distances,
        "cost_s": cost_s,
        "cost_l": cost_l,
        "facility_capacities": facility_capacities,
        "payload_small": payload_small,
        "payload_large": payload_large,
        "range_small": range_small,
        "range_large": range_large,
        "customer_weights": customer_weights  
    }

In [4]:
# Model initialization and solve
# Initialize the model
def solve_facility_problem(file_path):
    data = read_problem_instance(file_path)

    F = range(data['num_facilities'])
    C = range(data['num_customers'])
    Ds = range(30)  # S-type drones
    Dl = range(30)  # L-type drones

    model = pyomo.ConcreteModel()

    # Parameters
    d = data['distances']
    cs = data['cost_s']
    cl = data['cost_l']
    K = data['facility_capacities']
    Ps = data['payload_small']
    Pl = data['payload_large']
    Rs = data['range_small']
    Rl = data['range_large']
    W = data['customer_weights']

    ws = 1
    wl = 3

    # Decision Variables
    model.x = pyomo.Var(F, domain=pyomo.Binary)
    model.ys = pyomo.Var(Ds, F, C, domain=pyomo.Binary)
    model.yl = pyomo.Var(Dl, F, C, domain=pyomo.Binary)

    # Objective Function
    def objective_function_rule(model):
        fixed_costs = sum(K[i] * model.x[i] for i in F)
        variable_costs = sum(cs[i][j] * model.ys[s, i, j] + cl[i][j] * model.yl[l, i, j] \
                            for s in Ds for l in Dl for i in F for j in C)
        return fixed_costs + variable_costs

    model.objective = pyomo.Objective(rule=objective_function_rule, sense=pyomo.minimize)

    # Constraints
    # 1. Each customer must be served by one facility and one type of drone.
    def is_customer_served(model, j):
        return sum(model.ys[s, i, j] for s in Ds for i in F) + sum(model.yl[l, i, j] for l in Dl for i in F) == 1

    model.customer_served = pyomo.Constraint(C, rule=is_customer_served)

    # 2. Each facility has a total capacity limit for the number of drones it can support.
    def is_facility_maxed(model, i):
        return sum(ws * model.ys[s, i, j] for s in Ds for j in C) + sum(wl * model.yl[l, i, j] for l in Dl for j in C) <= K[i] * model.x[i]

    model.facility_capacity = pyomo.Constraint(F, rule=is_facility_maxed)

    # 3. A customer can only be assigned to a facility that is open. 
    def is_small_facility_open(model, s, i, j):
        return model.ys[s, i, j] <= model.x[i]

    model.small_facility_open = pyomo.Constraint(Ds, F, C, rule=is_small_facility_open)

    def is_large_facility_open(model, l, i, j):
        return model.yl[l, i, j] <= model.x[i]

    model.large_facility_open = pyomo.Constraint(Dl, F, C, rule=is_large_facility_open)

    # 4. The assigned drone type must be able to carry the package weight.
    def payload_capacity_small_rule(model, s, i, j):
        return W[j] * model.ys[s, i, j] <= Ps

    model.payload_capacity_small = pyomo.Constraint(Ds, F, C, rule=payload_capacity_small_rule)

    def payload_capacity_large_rule(model, l, i, j):
        return W[j] * model.yl[l, i, j] <= Pl

    model.payload_capacity_large = pyomo.Constraint(Dl, F, C, rule=payload_capacity_large_rule)

    # 5. S-type drones can only serve customers within Rs while L-type drones can serve customers within Rl
    def range_limit_small_rule(model, s, i, j):
        return d[i][j] * model.ys[s, i, j] <= Rs

    model.range_limit_small = pyomo.Constraint(Ds, F, C, rule=range_limit_small_rule)

    def range_limit_large_rule(model, l, i, j):
        return d[i][j] * model.yl[l, i, j] <= Rl

    model.range_limit_large = pyomo.Constraint(Dl, F, C, rule=range_limit_large_rule)


    # # 6. Each small drone should be assigned to exactly one customer
    # def distinct_drone_assignment_small_rule(model, s):
    #     return sum(model.ys[s, i, j] for i in F for j in C) == 1

    # model.distinct_drone_assignment_small = pyomo.Constraint(Ds, rule=distinct_drone_assignment_small_rule)

    # # Each large drone should be assigned to exactly one customer
    # def distinct_drone_assignment_large_rule(model, l):
    #     return sum(model.yl[l, i, j] for i in F for j in C) == 1

    # model.distinct_drone_assignment_large = pyomo.Constraint(Dl, rule=distinct_drone_assignment_large_rule)

    # # 7. Each drone should be assigned to exactly one facility
    # def distinct_facility_assignment_small_rule(model, s):
    #     return sum(model.ys[s, i, j] for i in F for j in C) == 1

    # model.distinct_facility_assignment_small = pyomo.Constraint(Ds, rule=distinct_facility_assignment_small_rule)

    # def distinct_facility_assignment_large_rule(model, l):
    #     return sum(model.yl[l, i, j] for i in F for j in C) == 1

    # model.distinct_facility_assignment_large = pyomo.Constraint(Dl, rule=distinct_facility_assignment_large_rule)

    # Solve the model
    solver = pyomo.SolverFactory('gurobi')
    result = solver.solve(model, tee=True)

    # Check if the model is infeasible
    if result.solver.termination_condition == pyomo.TerminationCondition.infeasible:
        print("The model is infeasible.")
        return None  # Return None if the problem is infeasible

    # Print Results if the model is feasible
    print("-----Facility Openings-----")
    for i in F:
        print(f"Facility {i}: {'Open' if model.x[i]() > 0.5 else 'Closed'}")

    print("-----Customer Assignments-----")
    for j in C:
        assigned = False
        for i in F:
            for s in Ds:
                if model.ys[s, i, j]() > 0.5:
                    print(f"Customer {j} is served by Facility {i} using Small Drone {s}.")
                    assigned = True
            for l in Dl:
                if model.yl[l, i, j]() > 0.5:
                    print(f"Customer {j} is served by Facility {i} using Large Drone {l}.")
                    assigned = True
        if not assigned:
            print(f"Customer {j} is not assigned to any facility.")

    print(f"Total Cost: {model.objective()}")

In [5]:
for i in range(10):
    problem_file = f"generated_problems/class_1/problem_instance_{i+1}.txt"
    try:
        # Attempt to solve the facility problem
        result = solve_facility_problem(problem_file)

        # Check if result is infeasible or error occurred
        if result.solver.status == pyomo.opt.SolverStatus.error:
            print(f"Problem {i+1} encountered an error. Skipping it.\n")
        elif result.solver.status == pyomo.opt.SolverStatus.infeasible:
            print(f"Problem {i+1} is infeasible, skipping it.\n")
        else:
            # Continue processing if the result is valid
            print(f"Problem {i+1} solved successfully.\n")
            
            # Optionally print additional details
            print(f"Solver termination condition: {result.solver.termination_condition}")
            print(f"Solver message: {result.solver.message}")

    except Exception as e:
        # If an error occurs (e.g., solver fails), print the error and continue with the next problem
        print(f"Error solving problem {i+1}: {e}\n")


ERROR: Constructing component 'objective' from data=None failed:
        KeyboardInterrupt:


KeyboardInterrupt: 

In [19]:
solve_facility_problem("generated_problems/class_1/problem_instance_1.txt")

Set parameter Username
Set parameter LicenseID to value 2602449
Academic license - for non-commercial use only - expires 2025-12-20
Read LP format model from file C:\Users\ArisM\AppData\Local\Temp\tmps7wrxf_v.pyomo.lp
Reading time = 0.16 seconds
x1: 54040 rows, 18010 columns, 108010 nonzeros
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-4440 CPU @ 3.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 54040 rows, 18010 columns and 108010 nonzeros
Model fingerprint: 0xb69da930
Variable types: 0 continuous, 18010 integer (18010 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [5e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 36000 rows and 14280 columns
Presolve time: 0.02s

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.03 work units)
Thre